# Problem
The question we are trying to answear in this notebook is if deskewing images before applying HOG descriptor increases performance of Neural Network, SVM and RandomForest models.


The main task is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## [deskew]
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img
## [deskew]


In [3]:
def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()

In [4]:
def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

# Data preprocessing

Raw data

In [5]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

C:\Users\Szymon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
all_images = [train_images[i] for i in range(0,train_images.shape[0])] + [test_images[i] for i in range(0,test_images.shape[0])]

all_labels = list(train_labels) + list(test_labels)

print("Shape of all images: {}\nLen of all labels: {}".format(len(all_images), len(all_labels)))

Shape of all images: 70000
Len of all labels: 70000


Reducing training size to shorten learning time

In [7]:
from sklearn.model_selection import train_test_split
from collections import Counter

train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, train_size=0.014, random_state=42, stratify=all_labels)

print("Training size: {}\nTest size: {}".format(len(train_images), len(test_images)))
print("Class counter\nIn train data: {}\n In test data: {}".format(Counter(train_labels), Counter(test_labels)))

Training size: 980
Test size: 69020
Class counter
In train data: Counter({1: 110, 7: 102, 3: 100, 2: 98, 0: 97, 9: 97, 6: 96, 4: 96, 8: 96, 5: 88})
 In test data: Counter({1: 7767, 7: 7191, 3: 7041, 2: 6892, 9: 6861, 0: 6806, 6: 6780, 8: 6729, 4: 6728, 5: 6225})


C:\Users\Szymon\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Vectorised raw data

In [8]:
train_raw = np.array(train_images).reshape(len(train_images), 28 * 28)
test_raw = np.array(test_images).reshape(len(test_images), 28 * 28)

Deskewed data

In [9]:
train_deskewed = np.float32([deskew(im) for im in train_raw])
test_deskewed = np.float32([deskew(im) for im in test_raw])
train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)

Hog descriptor data

In [10]:
hogdata_train = np.float32([openCVHOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([openCVHOG(im) for im in test_images]).reshape(-1,81)

hogdata_train_deskewed = np.float32([openCVHOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([openCVHOG(deskew(im)) for im in test_images]).reshape(-1,81)

# Model SVM

Model selection

In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [31]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}

# for deskewed data
svc_deskewed = SVC(class_weight='balanced')
clf_d = GridSearchCV(svc_deskewed, parameters, scoring='accuracy', cv=StratifiedKFold())
clf_d.fit(hogdata_train_deskewed, train_labels)
params_d = clf_d.best_params_
my_svc_deskewed = clf_d.best_estimator_
print("Deskewed data:\nChosing params: ", params_d)

svc = SVC(class_weight='balanced')
clf = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
clf.fit(hogdata_train, train_labels)
params = clf.best_params_
my_svc = clf.best_estimator_
print("Non deskewed data:\nChosing params: ", params)

Deskewed data:
Chosing params:  {'C': 0.7041658291457287, 'kernel': 'linear'}
Non deskewed data:
Chosing params:  {'C': 1.357105527638191, 'kernel': 'linear'}


Training models with HOG descriptors with and without deskewing images

In [32]:
# model_deskewed = svm.SVC(C=15.5,gamma=0.7)
# model_non_deskewed = svm.SVC(C=15.5,gamma=0.7)

# model_deskewed.fit(hogdata_train_deskewed, train_labels)
# model_non_deskewed.fit(hogdata_train, train_labels)

pred_labels_deskewed = my_svc_deskewed.predict(hogdata_test_deskewed)
pred_labels_non_deskewed = my_svc.predict(hogdata_test)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy for deskewed images: {}\nAccuracy for non deskewed images: {}".\
      format(accuracy_score(test_labels, pred_labels_deskewed), accuracy_score(test_labels, pred_labels_non_deskewed)))

Accuracy for deskewed images: 0.9512170385395537
Accuracy for non deskewed images: 0.9326716893654013


In [34]:
cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)


print("Confusion matrix for deskewed images:\n{}\n\nConfusion matrix for non deskewed images:\n{}".\
      format(cm_deskewed, cm_non_deskewed))

Confusion matrix for deskewed images:
[[6598    3   14    4   38   22   44   20   10   53]
 [   2 7569   49   15   35    0   10   70    8    9]
 [  10   16 6511   29   33    8   17  151   88   29]
 [   2    4  137 6695    2   30    0   62   75   34]
 [  13    3    9    2 6478    3  102    4   48   66]
 [  19    3    8   37   34 5962   37    7   64   54]
 [  24   21    8    2   54   27 6622    0   16    6]
 [  70   10  175  175   20    3    1 6629   41   67]
 [  33   28   46   60   71  108   44   34 6132  173]
 [  55    3    5   21   41   63   11   91  114 6457]]

Confusion matrix for non deskewed images:
[[6609    3   18    2   28   22   70   11   16   27]
 [  19 7569   45   21   22    6    7   59   10    9]
 [  25   24 6355   59   25   20    8  261   92   23]
 [   7   14  120 6644    2   67    0   87   77   23]
 [  50   14   24   16 6416   10   64    6   59   69]
 [  47   52   13  109   27 5652   51   16  145  113]
 [  80   21    5    4  119   72 6419    0   53    7]
 [ 149   10  287 


*   **PRECISION** = TP / (TP+FP)
*   **RECALL** = TP + (TP+FN)
*   **F1 score** = 2*PRECISION*RECALL/(PRECISION+RECALL)
*   **ACCURACY** = SUM_OF_DIAGNONAL ELEMENTS/SUM OF ALL ELEMENTS
*   **Macro_AVG OF PRECISION** = SUM OF PRECISIONS/NUMBER OF CLASSES
*   **Weighted AVG OF PRECISION** = SUM OVER CLASSES PRECISION(CLASS)*WEIGHT*   (CLASS),
**WEIGHT** = CLASS SUPPORT/ALL ELEMENTS
*   **MICRO AVG OF PRECISION** = SUM (TP(CLASS))/SUM(TP(CLASS)+FP(CLASS))
   



# Model RandomForest

In [36]:
from sklearn.ensemble import RandomForestClassifier

Model selection

In [41]:
parameters = {'n_estimators':[i for i in range(80, 130)] , 'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

# for deskewed data
rf_deskewed = RandomForestClassifier(class_weight='balanced')
clf_d = GridSearchCV(rf_deskewed, parameters, scoring='accuracy', cv=StratifiedKFold())
clf_d.fit(hogdata_train_deskewed, train_labels)
params_d = clf_d.best_params_
my_rf_deskewed = clf_d.best_estimator_
print("Deskewed data:\nChosing params: ", params_d)

rf = RandomForestClassifier(class_weight='balanced')
clf = GridSearchCV(rf, parameters, scoring='accuracy', cv=StratifiedKFold())
clf.fit(hogdata_train, train_labels)
params = clf.best_params_
my_rf = clf.best_estimator_
print("Non deskewed data:\nChosing params: ", params)

Deskewed data:
Chosing params:  {'max_depth': None, 'n_estimators': 126}
Non deskewed data:
Chosing params:  {'max_depth': None, 'n_estimators': 114}


Training models with HOG descriptors with and without deskewing images

In [43]:
# model_deskewed = RandomForestClassifier(max_depth=15, n_estimators=100, max_features=60)
# model_non_deskewed = RandomForestClassifier(max_depth=15, n_estimators=100, max_features=60)

# model_deskewed.fit(hogdata_train_deskewed, train_labels)
# model_non_deskewed.fit(hogdata_train, train_labels)

pred_labels_deskewed = my_rf_deskewed.predict(hogdata_test_deskewed)
pred_labels_non_deskewed = my_rf.predict(hogdata_test)

In [44]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy for deskewed images: {}\nAccuracy for non deskewed images: {}".\
      format(accuracy_score(test_labels, pred_labels_deskewed), accuracy_score(test_labels, pred_labels_non_deskewed)))

Accuracy for deskewed images: 0.9345407128368589
Accuracy for non deskewed images: 0.9113735149232106


In [45]:
cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)


print("Confusion matrix for deskewed images:\n{}\n\nConfusion matrix for non deskewed images:\n{}".\
      format(cm_deskewed, cm_non_deskewed))

Confusion matrix for deskewed images:
[[6515    2   24    5   10  121   24   16   22   67]
 [   4 7561   45    8   48    0   11   85    2    3]
 [  10   22 6376   79   35   14   28  189  103   36]
 [   3    5  158 6641    5   48    2   81   73   25]
 [  20    6    4    3 6357    9  179    1   29  120]
 [  46   11    6   27   23 5887   60    9   96   60]
 [  76   21    5    0   56   46 6538    0   16   22]
 [ 133    5  151  209    9   24    0 6565   38   57]
 [  99   47   45   77   79  159   52   30 5851  290]
 [ 129    6    5   28   51  110   41  101  179 6211]]

Confusion matrix for non deskewed images:
[[6485    7   15    5   18  138   58   13   26   41]
 [  10 7548   48   16   25   10   10   95    1    4]
 [  45   33 5934  300   54   38   15  324  107   42]
 [   5    8  138 6572    6  101    2   92   85   32]
 [  51    7   10    0 6340   20  126    3   47  124]
 [  66   68    8   65   30 5610   65   28  222   63]
 [ 125   24    3    1  114  118 6301    0   64   30]
 [  75    4  201 

# Model Neural Network

In [112]:
from keras import models
from keras import layers

Data vectorisation (HOG on deskewed images and HOG on non deskewed images)

In [114]:
nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((len(hogdata_train_deskewed), 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((len(hogdata_test_deskewed), 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_non_deskewed = np.array(hogdata_train).reshape((len(hogdata_train), 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((len(hogdata_test), 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

In [115]:
from keras.utils import to_categorical

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

encoded_test_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [116]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from statistics import mean

Model selection

In [127]:
def build_model():
    network = models.Sequential()
    network.add(layers.Dense(512, activation='relu', input_shape=(81,)))
    network.add(layers.Dense(10, activation='softmax'))
    network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return network

In [128]:
def build_model_2():
    network = models.Sequential()
    network.add(layers.Dense(300, activation='relu', input_shape=(81,)))
    network.add(layers.Dense(300, activation='relu', input_shape=(300,)))

    network.add(layers.Dense(10, activation='softmax'))
    network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return network

In [138]:
def build_model_3():
    network = models.Sequential()
    network.add(layers.Dense(300, activation='relu', input_shape=(81,)))
    network.add(layers.Dense(300, activation='relu', input_shape=(300,)))
    network.add(layers.Dense(300, activation='relu', input_shape=(300,)))

    network.add(layers.Dense(10, activation='softmax'))
    network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return network

In [129]:
def evaluate_classifier(model_f, train_data, train_targets, epochs, **kwargs):
    k = 5
    num_val_samples = len(train_data) // k
    num_epochs = epochs
    all_scores = []
    for i in range(k):
        print('processing fold #', i)
        # Prepare the validation data: data from partition # k
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        # Prepare the training data: data from all other partitions
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)

        # Build the Keras model (already compiled)
        model = model_f()
        # Train the model (in silent mode, verbose=0)
        model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size=124, verbose=1)
        # Evaluate the model on the validation data
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
        all_scores.append(val_mae)
    return mean(all_scores)


model 1

In [133]:
# for deskewed
evaluate_classifier(build_model, nn_train_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 248us/step - loss: 2.3020 - accuracy: 0.1135
Epoch 2/250
784/784 [==============================] - 0s 33us/step - loss: 2.2992 - accuracy: 0.1212
Epoch 3/250
784/784 [==============================] - 0s 32us/step - loss: 2.2974 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 46us/step - loss: 2.2956 - accuracy: 0.1199
Epoch 5/250
784/784 [==============================] - 0s 48us/step - loss: 2.2941 - accuracy: 0.1199
Epoch 6/250
784/784 [==============================] - 0s 46us/step - loss: 2.2930 - accuracy: 0.1199
Epoch 7/250
784/784 [==============================] - 0s 36us/step - loss: 2.2911 - accuracy: 0.1199
Epoch 8/250
784/784 [==============================] - 0s 28us/step - loss: 2.2896 - accuracy: 0.1199
Epoch 9/250
784/784 [==============================] - 0s 29us/step - loss: 2.2877 - accuracy: 0.1199
Epoch 10/250
784/784 [==============================] - 0s 31

784/784 [==============================] - 0s 29us/step - loss: 1.3079 - accuracy: 0.7207
Epoch 157/250
784/784 [==============================] - 0s 29us/step - loss: 1.3023 - accuracy: 0.7181
Epoch 158/250
784/784 [==============================] - 0s 28us/step - loss: 1.2976 - accuracy: 0.7130
Epoch 159/250
784/784 [==============================] - 0s 28us/step - loss: 1.2910 - accuracy: 0.7423
Epoch 160/250
784/784 [==============================] - 0s 27us/step - loss: 1.2857 - accuracy: 0.7474
Epoch 161/250
784/784 [==============================] - 0s 28us/step - loss: 1.2808 - accuracy: 0.7143
Epoch 162/250
784/784 [==============================] - 0s 27us/step - loss: 1.2761 - accuracy: 0.7258
Epoch 163/250
784/784 [==============================] - 0s 29us/step - loss: 1.2693 - accuracy: 0.7436
Epoch 164/250
784/784 [==============================] - 0s 28us/step - loss: 1.2629 - accuracy: 0.7423
Epoch 165/250
784/784 [==============================] - 0s 27us/step - loss: 

784/784 [==============================] - 0s 31us/step - loss: 1.9679 - accuracy: 0.5153
Epoch 63/250
784/784 [==============================] - ETA: 0s - loss: 1.9421 - accuracy: 0.47 - 0s 32us/step - loss: 1.9599 - accuracy: 0.4260
Epoch 64/250
784/784 [==============================] - 0s 29us/step - loss: 1.9523 - accuracy: 0.5089
Epoch 65/250
784/784 [==============================] - 0s 33us/step - loss: 1.9423 - accuracy: 0.4885
Epoch 66/250
784/784 [==============================] - 0s 31us/step - loss: 1.9335 - accuracy: 0.5115
Epoch 67/250
784/784 [==============================] - 0s 27us/step - loss: 1.9247 - accuracy: 0.5064
Epoch 68/250
784/784 [==============================] - 0s 28us/step - loss: 1.9168 - accuracy: 0.4490
Epoch 69/250
784/784 [==============================] - 0s 29us/step - loss: 1.9070 - accuracy: 0.5102
Epoch 70/250
784/784 [==============================] - 0s 28us/step - loss: 1.8985 - accuracy: 0.4707
Epoch 71/250
784/784 [======================

784/784 [==============================] - 0s 31us/step - loss: 1.3911 - accuracy: 0.6798
Epoch 141/250
784/784 [==============================] - 0s 32us/step - loss: 1.3843 - accuracy: 0.6722
Epoch 142/250
784/784 [==============================] - 0s 29us/step - loss: 1.3782 - accuracy: 0.6849
Epoch 143/250
784/784 [==============================] - 0s 28us/step - loss: 1.3715 - accuracy: 0.6671
Epoch 144/250
784/784 [==============================] - 0s 28us/step - loss: 1.3674 - accuracy: 0.6901
Epoch 145/250
784/784 [==============================] - 0s 28us/step - loss: 1.3610 - accuracy: 0.7041
Epoch 146/250
784/784 [==============================] - 0s 30us/step - loss: 1.3545 - accuracy: 0.6875
Epoch 147/250
784/784 [==============================] - 0s 31us/step - loss: 1.3504 - accuracy: 0.6760
Epoch 148/250
784/784 [==============================] - 0s 29us/step - loss: 1.3451 - accuracy: 0.6747
Epoch 149/250
784/784 [==============================] - 0s 28us/step - loss: 

784/784 [==============================] - 0s 33us/step - loss: 2.1405 - accuracy: 0.3648
Epoch 45/250
784/784 [==============================] - 0s 31us/step - loss: 2.1335 - accuracy: 0.4018
Epoch 46/250
784/784 [==============================] - 0s 32us/step - loss: 2.1264 - accuracy: 0.3724
Epoch 47/250
784/784 [==============================] - 0s 29us/step - loss: 2.1204 - accuracy: 0.4592
Epoch 48/250
784/784 [==============================] - 0s 32us/step - loss: 2.1139 - accuracy: 0.4260
Epoch 49/250
784/784 [==============================] - 0s 27us/step - loss: 2.1053 - accuracy: 0.4069
Epoch 50/250
784/784 [==============================] - 0s 29us/step - loss: 2.0983 - accuracy: 0.4145
Epoch 51/250
784/784 [==============================] - 0s 28us/step - loss: 2.0907 - accuracy: 0.3890
Epoch 52/250
784/784 [==============================] - 0s 28us/step - loss: 2.0831 - accuracy: 0.3929
Epoch 53/250
784/784 [==============================] - 0s 29us/step - loss: 2.0751 - 

...

784/784 [==============================] - 0s 32us/step - loss: 1.6845 - accuracy: 0.5842
Epoch 105/250
784/784 [==============================] - 0s 31us/step - loss: 1.6778 - accuracy: 0.5651
Epoch 106/250
784/784 [==============================] - 0s 34us/step - loss: 1.6713 - accuracy: 0.5791
Epoch 107/250
784/784 [==============================] - 0s 32us/step - loss: 1.6649 - accuracy: 0.5829
Epoch 108/250
784/784 [==============================] - 0s 32us/step - loss: 1.6574 - accuracy: 0.5548
Epoch 109/250
784/784 [==============================] - ETA: 0s - loss: 1.7346 - accuracy: 0.54 - 0s 32us/step - loss: 1.6526 - accuracy: 0.5599
Epoch 110/250
784/784 [==============================] - 0s 33us/step - loss: 1.6445 - accuracy: 0.5753
Epoch 111/250
784/784 [==============================] - 0s 27us/step - loss: 1.6379 - accuracy: 0.5855
Epoch 112/250
784/784 [==============================] - 0s 29us/step - loss: 1.6319 - accuracy: 0.5893
Epoch 113/250
784/784 [=============

784/784 [==============================] - 0s 31us/step - loss: 2.2972 - accuracy: 0.1696
Epoch 6/250
784/784 [==============================] - 0s 36us/step - loss: 2.2962 - accuracy: 0.2105
Epoch 7/250
784/784 [==============================] - 0s 33us/step - loss: 2.2950 - accuracy: 0.1747
Epoch 8/250
784/784 [==============================] - 0s 33us/step - loss: 2.2936 - accuracy: 0.2309
Epoch 9/250
784/784 [==============================] - 0s 33us/step - loss: 2.2921 - accuracy: 0.2054
Epoch 10/250
784/784 [==============================] - 0s 33us/step - loss: 2.2908 - accuracy: 0.2054
Epoch 11/250
784/784 [==============================] - 0s 32us/step - loss: 2.2888 - accuracy: 0.2092
Epoch 12/250
784/784 [==============================] - 0s 32us/step - loss: 2.2867 - accuracy: 0.2054
Epoch 13/250
784/784 [==============================] - 0s 34us/step - loss: 2.2849 - accuracy: 0.2156
Epoch 14/250
784/784 [==============================] - 0s 30us/step - loss: 2.2830 - accu

784/784 [==============================] - 0s 33us/step - loss: 1.8783 - accuracy: 0.4987
Epoch 84/250
784/784 [==============================] - 0s 30us/step - loss: 1.8709 - accuracy: 0.5166
Epoch 85/250
784/784 [==============================] - 0s 31us/step - loss: 1.8631 - accuracy: 0.4898
Epoch 86/250
784/784 [==============================] - 0s 32us/step - loss: 1.8551 - accuracy: 0.5051
Epoch 87/250
784/784 [==============================] - 0s 31us/step - loss: 1.8479 - accuracy: 0.5026
Epoch 88/250
784/784 [==============================] - 0s 29us/step - loss: 1.8394 - accuracy: 0.5140
Epoch 89/250
784/784 [==============================] - 0s 28us/step - loss: 1.8333 - accuracy: 0.5293
Epoch 90/250
784/784 [==============================] - 0s 31us/step - loss: 1.8254 - accuracy: 0.5013
Epoch 91/250
784/784 [==============================] - 0s 28us/step - loss: 1.8180 - accuracy: 0.5013
Epoch 92/250
784/784 [==============================] - 0s 29us/step - loss: 1.8118 - 

784/784 [==============================] - 0s 29us/step - loss: 1.0501 - accuracy: 0.7742
Epoch 239/250
784/784 [==============================] - ETA: 0s - loss: 1.0786 - accuracy: 0.75 - 0s 29us/step - loss: 1.0472 - accuracy: 0.7704
Epoch 240/250
784/784 [==============================] - 0s 29us/step - loss: 1.0421 - accuracy: 0.7704
Epoch 241/250
784/784 [==============================] - 0s 33us/step - loss: 1.0399 - accuracy: 0.7781
Epoch 242/250
784/784 [==============================] - 0s 33us/step - loss: 1.0338 - accuracy: 0.7857
Epoch 243/250
784/784 [==============================] - 0s 32us/step - loss: 1.0309 - accuracy: 0.7870
Epoch 244/250
784/784 [==============================] - 0s 29us/step - loss: 1.0289 - accuracy: 0.7781
Epoch 245/250
784/784 [==============================] - 0s 32us/step - loss: 1.0258 - accuracy: 0.7755
Epoch 246/250
784/784 [==============================] - 0s 32us/step - loss: 1.0192 - accuracy: 0.7793
Epoch 247/250
784/784 [=============

0.7755101919174194

In [135]:
# for non deskewed 
evaluate_classifier(build_model, nn_train_non_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 270us/step - loss: 2.3022 - accuracy: 0.1518
Epoch 2/250
784/784 [==============================] - 0s 32us/step - loss: 2.3002 - accuracy: 0.1288
Epoch 3/250
784/784 [==============================] - 0s 33us/step - loss: 2.2985 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 36us/step - loss: 2.2970 - accuracy: 0.1199
Epoch 5/250
784/784 [==============================] - 0s 37us/step - loss: 2.2961 - accuracy: 0.1199
Epoch 6/250
784/784 [==============================] - 0s 33us/step - loss: 2.2944 - accuracy: 0.1199
Epoch 7/250
784/784 [==============================] - 0s 36us/step - loss: 2.2931 - accuracy: 0.1199
Epoch 8/250
784/784 [==============================] - 0s 37us/step - loss: 2.2917 - accuracy: 0.1199
Epoch 9/250
784/784 [==============================] - 0s 37us/step - loss: 2.2901 - accuracy: 0.1199
Epoch 10/250
784/784 [==============================] - 0s 38

784/784 [==============================] - 0s 29us/step - loss: 1.4556 - accuracy: 0.6594
Epoch 156/250
784/784 [==============================] - 0s 30us/step - loss: 1.4509 - accuracy: 0.6696
Epoch 157/250
784/784 [==============================] - 0s 27us/step - loss: 1.4440 - accuracy: 0.6582
Epoch 158/250
784/784 [==============================] - 0s 28us/step - loss: 1.4400 - accuracy: 0.6837
Epoch 159/250
784/784 [==============================] - 0s 28us/step - loss: 1.4344 - accuracy: 0.6543
Epoch 160/250
784/784 [==============================] - 0s 27us/step - loss: 1.4315 - accuracy: 0.6594
Epoch 161/250
784/784 [==============================] - 0s 29us/step - loss: 1.4250 - accuracy: 0.6658
Epoch 162/250
784/784 [==============================] - 0s 28us/step - loss: 1.4202 - accuracy: 0.6633
Epoch 163/250
784/784 [==============================] - 0s 28us/step - loss: 1.4139 - accuracy: 0.6722
Epoch 164/250
784/784 [==============================] - 0s 28us/step - loss: 

784/784 [==============================] - 0s 34us/step - loss: 2.0847 - accuracy: 0.3367
Epoch 62/250
784/784 [==============================] - 0s 29us/step - loss: 2.0781 - accuracy: 0.3559
Epoch 63/250
784/784 [==============================] - 0s 36us/step - loss: 2.0720 - accuracy: 0.3163
Epoch 64/250
784/784 [==============================] - 0s 31us/step - loss: 2.0661 - accuracy: 0.3380
Epoch 65/250
784/784 [==============================] - 0s 31us/step - loss: 2.0582 - accuracy: 0.3291
Epoch 66/250
784/784 [==============================] - 0s 29us/step - loss: 2.0518 - accuracy: 0.3291
Epoch 67/250
784/784 [==============================] - 0s 33us/step - loss: 2.0460 - accuracy: 0.3112
Epoch 68/250
784/784 [==============================] - 0s 32us/step - loss: 2.0405 - accuracy: 0.3750
Epoch 69/250
784/784 [==============================] - 0s 31us/step - loss: 2.0323 - accuracy: 0.3610
Epoch 70/250
784/784 [==============================] - 0s 28us/step - loss: 2.0249 - 

784/784 [==============================] - 0s 32us/step - loss: 1.2539 - accuracy: 0.6964
Epoch 215/250
784/784 [==============================] - 0s 33us/step - loss: 1.2481 - accuracy: 0.6901
Epoch 216/250
784/784 [==============================] - 0s 34us/step - loss: 1.2441 - accuracy: 0.7028
Epoch 217/250
784/784 [==============================] - 0s 38us/step - loss: 1.2400 - accuracy: 0.7028
Epoch 218/250
784/784 [==============================] - 0s 31us/step - loss: 1.2362 - accuracy: 0.7066
Epoch 219/250
784/784 [==============================] - 0s 32us/step - loss: 1.2319 - accuracy: 0.7054
Epoch 220/250
784/784 [==============================] - 0s 31us/step - loss: 1.2283 - accuracy: 0.7028
Epoch 221/250
784/784 [==============================] - 0s 28us/step - loss: 1.2232 - accuracy: 0.6939
Epoch 222/250
784/784 [==============================] - 0s 31us/step - loss: 1.2210 - accuracy: 0.7105
Epoch 223/250
784/784 [==============================] - 0s 31us/step - loss: 

784/784 [==============================] - 0s 22us/step - loss: 1.7074 - accuracy: 0.5931
Epoch 120/250
784/784 [==============================] - 0s 20us/step - loss: 1.7008 - accuracy: 0.6020
Epoch 121/250
784/784 [==============================] - 0s 20us/step - loss: 1.6942 - accuracy: 0.5663
Epoch 122/250
784/784 [==============================] - 0s 25us/step - loss: 1.6874 - accuracy: 0.5829
Epoch 123/250
784/784 [==============================] - 0s 23us/step - loss: 1.6809 - accuracy: 0.6122
Epoch 124/250
784/784 [==============================] - 0s 22us/step - loss: 1.6755 - accuracy: 0.5969
Epoch 125/250
784/784 [==============================] - 0s 22us/step - loss: 1.6714 - accuracy: 0.5906
Epoch 126/250
784/784 [==============================] - 0s 22us/step - loss: 1.6631 - accuracy: 0.6059
Epoch 127/250
784/784 [==============================] - 0s 22us/step - loss: 1.6571 - accuracy: 0.5816
Epoch 128/250
784/784 [==============================] - 0s 22us/step - loss: 

...

784/784 [==============================] - 0s 31us/step - loss: 1.3028 - accuracy: 0.6888
Epoch 182/250
784/784 [==============================] - 0s 29us/step - loss: 1.2953 - accuracy: 0.7041
Epoch 183/250
784/784 [==============================] - 0s 29us/step - loss: 1.2920 - accuracy: 0.7054
Epoch 184/250
784/784 [==============================] - 0s 29us/step - loss: 1.2854 - accuracy: 0.7015
Epoch 185/250
784/784 [==============================] - 0s 28us/step - loss: 1.2813 - accuracy: 0.7015
Epoch 186/250
784/784 [==============================] - 0s 29us/step - loss: 1.2774 - accuracy: 0.6888
Epoch 187/250
784/784 [==============================] - 0s 29us/step - loss: 1.2745 - accuracy: 0.7015
Epoch 188/250
784/784 [==============================] - 0s 28us/step - loss: 1.2690 - accuracy: 0.6964
Epoch 189/250
784/784 [==============================] - 0s 33us/step - loss: 1.2649 - accuracy: 0.7079
Epoch 190/250
784/784 [==============================] - 0s 32us/step - loss: 

784/784 [==============================] - 0s 31us/step - loss: 2.2931 - accuracy: 0.2041
Epoch 9/250
784/784 [==============================] - 0s 29us/step - loss: 2.2915 - accuracy: 0.1594
Epoch 10/250
784/784 [==============================] - 0s 29us/step - loss: 2.2900 - accuracy: 0.2130
Epoch 11/250
784/784 [==============================] - 0s 30us/step - loss: 2.2884 - accuracy: 0.2411
Epoch 12/250
784/784 [==============================] - 0s 29us/step - loss: 2.2864 - accuracy: 0.2883
Epoch 13/250
784/784 [==============================] - 0s 29us/step - loss: 2.2842 - accuracy: 0.1875
Epoch 14/250
784/784 [==============================] - 0s 29us/step - loss: 2.2819 - accuracy: 0.2041
Epoch 15/250
784/784 [==============================] - 0s 29us/step - loss: 2.2798 - accuracy: 0.2602
Epoch 16/250
784/784 [==============================] - 0s 28us/step - loss: 2.2775 - accuracy: 0.2092
Epoch 17/250
784/784 [==============================] - 0s 27us/step - loss: 2.2751 - a

784/784 [==============================] - 0s 29us/step - loss: 1.3673 - accuracy: 0.6786
Epoch 165/250
784/784 [==============================] - 0s 29us/step - loss: 1.3619 - accuracy: 0.6824
Epoch 166/250
784/784 [==============================] - 0s 30us/step - loss: 1.3584 - accuracy: 0.6798
Epoch 167/250
784/784 [==============================] - 0s 31us/step - loss: 1.3511 - accuracy: 0.6773
Epoch 168/250
784/784 [==============================] - 0s 31us/step - loss: 1.3479 - accuracy: 0.6811
Epoch 169/250
784/784 [==============================] - 0s 28us/step - loss: 1.3430 - accuracy: 0.6901
Epoch 170/250
784/784 [==============================] - 0s 32us/step - loss: 1.3368 - accuracy: 0.6926
Epoch 171/250
784/784 [==============================] - 0s 28us/step - loss: 1.3348 - accuracy: 0.6862
Epoch 172/250
784/784 [==============================] - 0s 31us/step - loss: 1.3276 - accuracy: 0.6913
Epoch 173/250
784/784 [==============================] - 0s 28us/step - loss: 

784/784 [==============================] - 0s 29us/step - loss: 1.0388 - accuracy: 0.7449
Epoch 243/250
784/784 [==============================] - 0s 31us/step - loss: 1.0334 - accuracy: 0.7462
Epoch 244/250
784/784 [==============================] - 0s 31us/step - loss: 1.0312 - accuracy: 0.7398
Epoch 245/250
784/784 [==============================] - 0s 28us/step - loss: 1.0279 - accuracy: 0.7423
Epoch 246/250
784/784 [==============================] - 0s 33us/step - loss: 1.0226 - accuracy: 0.7513
Epoch 247/250
784/784 [==============================] - 0s 31us/step - loss: 1.0195 - accuracy: 0.7513
Epoch 248/250
784/784 [==============================] - 0s 31us/step - loss: 1.0173 - accuracy: 0.7487
Epoch 249/250
784/784 [==============================] - 0s 30us/step - loss: 1.0148 - accuracy: 0.7449
Epoch 250/250
196/196 [==============================] - 0s 860us/step


0.7173469424247741

Model 2

In [136]:
# for deskewed
evaluate_classifier(build_model_2, nn_train_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 459us/step - loss: 2.3020 - accuracy: 0.1033
Epoch 2/250
784/784 [==============================] - 0s 48us/step - loss: 2.2994 - accuracy: 0.1288
Epoch 3/250
784/784 [==============================] - 0s 43us/step - loss: 2.2976 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 38us/step - loss: 2.2952 - accuracy: 0.1199
Epoch 5/250
784/784 [==============================] - 0s 42us/step - loss: 2.2918 - accuracy: 0.1199
Epoch 6/250
784/784 [==============================] - 0s 36us/step - loss: 2.2887 - accuracy: 0.1199
Epoch 7/250
784/784 [==============================] - 0s 37us/step - loss: 2.2834 - accuracy: 0.1199
Epoch 8/250
784/784 [==============================] - 0s 36us/step - loss: 2.2769 - accuracy: 0.1390
Epoch 9/250
784/784 [==============================] - 0s 38us/step - loss: 2.2696 - accuracy: 0.1212
Epoch 10/250
784/784 [==============================] - ETA: 

784/784 [==============================] - 0s 37us/step - loss: 0.8472 - accuracy: 0.7474
Epoch 155/250
784/784 [==============================] - 0s 34us/step - loss: 0.8489 - accuracy: 0.7398
Epoch 156/250
784/784 [==============================] - 0s 36us/step - loss: 0.8675 - accuracy: 0.7143
Epoch 157/250
784/784 [==============================] - 0s 37us/step - loss: 0.8317 - accuracy: 0.7474
Epoch 158/250
784/784 [==============================] - 0s 38us/step - loss: 0.8502 - accuracy: 0.7245
Epoch 159/250
784/784 [==============================] - 0s 37us/step - loss: 0.8156 - accuracy: 0.7602
Epoch 160/250
784/784 [==============================] - 0s 39us/step - loss: 0.8203 - accuracy: 0.7449
Epoch 161/250
784/784 [==============================] - 0s 41us/step - loss: 0.8202 - accuracy: 0.7538
Epoch 162/250
784/784 [==============================] - 0s 38us/step - loss: 0.8361 - accuracy: 0.7462
Epoch 163/250
784/784 [==============================] - 0s 39us/step - loss: 

784/784 [==============================] - 0s 34us/step - loss: 1.4844 - accuracy: 0.4872
Epoch 58/250
784/784 [==============================] - 0s 36us/step - loss: 1.4682 - accuracy: 0.4962
Epoch 59/250
784/784 [==============================] - 0s 34us/step - loss: 1.4562 - accuracy: 0.4872
Epoch 60/250
784/784 [==============================] - 0s 33us/step - loss: 1.4465 - accuracy: 0.4885
Epoch 61/250
784/784 [==============================] - 0s 32us/step - loss: 1.4414 - accuracy: 0.4898
Epoch 62/250
784/784 [==============================] - 0s 30us/step - loss: 1.4360 - accuracy: 0.4949
Epoch 63/250
784/784 [==============================] - 0s 32us/step - loss: 1.4176 - accuracy: 0.5089
Epoch 64/250
784/784 [==============================] - 0s 32us/step - loss: 1.4004 - accuracy: 0.5038
Epoch 65/250
784/784 [==============================] - 0s 32us/step - loss: 1.4207 - accuracy: 0.4962
Epoch 66/250
784/784 [==============================] - 0s 32us/step - loss: 1.3939 - 

784/784 [==============================] - 0s 36us/step - loss: 0.7348 - accuracy: 0.7691
Epoch 212/250
784/784 [==============================] - 0s 36us/step - loss: 0.7547 - accuracy: 0.7538
Epoch 213/250
784/784 [==============================] - 0s 33us/step - loss: 0.7240 - accuracy: 0.7500
Epoch 214/250
784/784 [==============================] - 0s 33us/step - loss: 0.7332 - accuracy: 0.7628
Epoch 215/250
784/784 [==============================] - 0s 34us/step - loss: 0.7407 - accuracy: 0.7526
Epoch 216/250
784/784 [==============================] - 0s 33us/step - loss: 0.7425 - accuracy: 0.7628
Epoch 217/250
784/784 [==============================] - 0s 32us/step - loss: 0.7247 - accuracy: 0.7602
Epoch 218/250
784/784 [==============================] - 0s 32us/step - loss: 0.7308 - accuracy: 0.7538
Epoch 219/250
784/784 [==============================] - 0s 31us/step - loss: 0.7159 - accuracy: 0.7487
Epoch 220/250
784/784 [==============================] - 0s 29us/step - loss: 

784/784 [==============================] - 0s 33us/step - loss: 1.0504 - accuracy: 0.6059
Epoch 117/250
784/784 [==============================] - 0s 31us/step - loss: 1.0625 - accuracy: 0.6212
Epoch 118/250
784/784 [==============================] - 0s 32us/step - loss: 1.0407 - accuracy: 0.6276
Epoch 119/250
784/784 [==============================] - 0s 32us/step - loss: 1.0503 - accuracy: 0.6339
Epoch 120/250
784/784 [==============================] - 0s 32us/step - loss: 1.0565 - accuracy: 0.6135
Epoch 121/250
784/784 [==============================] - 0s 31us/step - loss: 1.0523 - accuracy: 0.6071
Epoch 122/250
784/784 [==============================] - 0s 32us/step - loss: 1.0393 - accuracy: 0.6327
Epoch 123/250
784/784 [==============================] - 0s 33us/step - loss: 1.0257 - accuracy: 0.6352
Epoch 124/250
784/784 [==============================] - 0s 31us/step - loss: 1.0420 - accuracy: 0.6403
Epoch 125/250
784/784 [==============================] - ETA: 0s - loss: 0.994

784/784 [==============================] - 0s 37us/step - loss: 2.0639 - accuracy: 0.2602
Epoch 21/250
784/784 [==============================] - 0s 29us/step - loss: 2.0418 - accuracy: 0.2564
Epoch 22/250
784/784 [==============================] - 0s 33us/step - loss: 2.0178 - accuracy: 0.2895
Epoch 23/250
784/784 [==============================] - 0s 34us/step - loss: 1.9984 - accuracy: 0.2819
Epoch 24/250
784/784 [==============================] - 0s 34us/step - loss: 1.9797 - accuracy: 0.3151
Epoch 25/250
784/784 [==============================] - 0s 33us/step - loss: 1.9586 - accuracy: 0.3048
Epoch 26/250
784/784 [==============================] - 0s 33us/step - loss: 1.9379 - accuracy: 0.3329
Epoch 27/250
784/784 [==============================] - 0s 34us/step - loss: 1.9190 - accuracy: 0.3316
Epoch 28/250
784/784 [==============================] - 0s 33us/step - loss: 1.9063 - accuracy: 0.3189
Epoch 29/250
784/784 [==============================] - 0s 32us/step - loss: 1.8780 - 

784/784 [==============================] - 0s 29us/step - loss: 0.8856 - accuracy: 0.6952
Epoch 176/250
784/784 [==============================] - 0s 28us/step - loss: 0.9229 - accuracy: 0.6633
Epoch 177/250
784/784 [==============================] - 0s 29us/step - loss: 0.8932 - accuracy: 0.6747
Epoch 178/250
784/784 [==============================] - 0s 33us/step - loss: 0.8808 - accuracy: 0.6952
Epoch 179/250
784/784 [==============================] - 0s 46us/step - loss: 0.8888 - accuracy: 0.6811
Epoch 180/250
784/784 [==============================] - 0s 38us/step - loss: 0.8856 - accuracy: 0.6811
Epoch 181/250
784/784 [==============================] - 0s 45us/step - loss: 0.8742 - accuracy: 0.6939
Epoch 182/250
784/784 [==============================] - 0s 32us/step - loss: 0.8662 - accuracy: 0.7015
Epoch 183/250
784/784 [==============================] - 0s 34us/step - loss: 0.8987 - accuracy: 0.6786
Epoch 184/250
784/784 [==============================] - 0s 43us/step - loss: 

784/784 [==============================] - 0s 31us/step - loss: 1.1765 - accuracy: 0.6173
Epoch 81/250
784/784 [==============================] - 0s 28us/step - loss: 1.1564 - accuracy: 0.5842
Epoch 82/250
784/784 [==============================] - 0s 25us/step - loss: 1.1439 - accuracy: 0.6161
Epoch 83/250
784/784 [==============================] - 0s 31us/step - loss: 1.1433 - accuracy: 0.6046
Epoch 84/250
784/784 [==============================] - 0s 33us/step - loss: 1.1375 - accuracy: 0.6059
Epoch 85/250
784/784 [==============================] - 0s 36us/step - loss: 1.1373 - accuracy: 0.6084
Epoch 86/250
784/784 [==============================] - ETA: 0s - loss: 1.1909 - accuracy: 0.53 - 0s 37us/step - loss: 1.1264 - accuracy: 0.6161
Epoch 87/250
784/784 [==============================] - ETA: 0s - loss: 1.2355 - accuracy: 0.50 - 0s 34us/step - loss: 1.1247 - accuracy: 0.6097
Epoch 88/250
784/784 [==============================] - 0s 37us/step - loss: 1.1079 - accuracy: 0.6186
Ep

784/784 [==============================] - 0s 27us/step - loss: 0.5436 - accuracy: 0.8316
Epoch 236/250
784/784 [==============================] - 0s 25us/step - loss: 0.5890 - accuracy: 0.8036
Epoch 237/250
784/784 [==============================] - 0s 24us/step - loss: 0.5689 - accuracy: 0.8265
Epoch 238/250
784/784 [==============================] - 0s 25us/step - loss: 0.5650 - accuracy: 0.8380
Epoch 239/250
784/784 [==============================] - 0s 24us/step - loss: 0.5628 - accuracy: 0.8227
Epoch 240/250
784/784 [==============================] - 0s 25us/step - loss: 0.5526 - accuracy: 0.8265
Epoch 241/250
784/784 [==============================] - 0s 25us/step - loss: 0.5357 - accuracy: 0.8393
Epoch 242/250
784/784 [==============================] - 0s 28us/step - loss: 0.5518 - accuracy: 0.8278
Epoch 243/250
784/784 [==============================] - 0s 31us/step - loss: 0.5620 - accuracy: 0.8253
Epoch 244/250
784/784 [==============================] - 0s 27us/step - loss: 

0.7775510311126709

In [137]:
# for non deskewed
evaluate_classifier(build_model_2, nn_train_non_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 371us/step - loss: 2.3030 - accuracy: 0.1135
Epoch 2/250
784/784 [==============================] - 0s 28us/step - loss: 2.3004 - accuracy: 0.1084
Epoch 3/250
784/784 [==============================] - 0s 28us/step - loss: 2.2984 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 27us/step - loss: 2.2965 - accuracy: 0.1199
Epoch 5/250
784/784 [==============================] - 0s 25us/step - loss: 2.2945 - accuracy: 0.1658
Epoch 6/250
784/784 [==============================] - 0s 25us/step - loss: 2.2906 - accuracy: 0.1199
Epoch 7/250
784/784 [==============================] - 0s 24us/step - loss: 2.2878 - accuracy: 0.1224
Epoch 8/250
784/784 [==============================] - 0s 25us/step - loss: 2.2823 - accuracy: 0.1224
Epoch 9/250
784/784 [==============================] - 0s 28us/step - loss: 2.2762 - accuracy: 0.1620
Epoch 10/250
784/784 [==============================] - 0s 32

784/784 [==============================] - 0s 27us/step - loss: 0.9949 - accuracy: 0.6135
Epoch 160/250
784/784 [==============================] - 0s 24us/step - loss: 0.9876 - accuracy: 0.6071
Epoch 161/250
784/784 [==============================] - 0s 25us/step - loss: 0.9836 - accuracy: 0.6276
Epoch 162/250
784/784 [==============================] - 0s 24us/step - loss: 1.0176 - accuracy: 0.5957
Epoch 163/250
784/784 [==============================] - 0s 24us/step - loss: 0.9912 - accuracy: 0.6416
Epoch 164/250
784/784 [==============================] - 0s 25us/step - loss: 0.9696 - accuracy: 0.6224
Epoch 165/250
784/784 [==============================] - 0s 24us/step - loss: 0.9967 - accuracy: 0.6110
Epoch 166/250
784/784 [==============================] - 0s 24us/step - loss: 0.9885 - accuracy: 0.6135
Epoch 167/250
784/784 [==============================] - 0s 25us/step - loss: 0.9848 - accuracy: 0.6237
Epoch 168/250
784/784 [==============================] - 0s 25us/step - loss: 

784/784 [==============================] - 0s 27us/step - loss: 1.5252 - accuracy: 0.4643
Epoch 66/250
784/784 [==============================] - 0s 27us/step - loss: 1.5259 - accuracy: 0.4630
Epoch 67/250
784/784 [==============================] - 0s 27us/step - loss: 1.4960 - accuracy: 0.4962
Epoch 68/250
784/784 [==============================] - 0s 28us/step - loss: 1.5010 - accuracy: 0.4745
Epoch 69/250
784/784 [==============================] - 0s 27us/step - loss: 1.4841 - accuracy: 0.4592
Epoch 70/250
784/784 [==============================] - 0s 28us/step - loss: 1.4675 - accuracy: 0.4821
Epoch 71/250
784/784 [==============================] - 0s 28us/step - loss: 1.4595 - accuracy: 0.4732
Epoch 72/250
784/784 [==============================] - 0s 28us/step - loss: 1.4496 - accuracy: 0.5077
Epoch 73/250
784/784 [==============================] - 0s 28us/step - loss: 1.4428 - accuracy: 0.4885
Epoch 74/250
784/784 [==============================] - 0s 31us/step - loss: 1.4409 - 

784/784 [==============================] - 0s 28us/step - loss: 0.8804 - accuracy: 0.6837
Epoch 221/250
784/784 [==============================] - 0s 28us/step - loss: 0.8826 - accuracy: 0.6786
Epoch 222/250
784/784 [==============================] - 0s 29us/step - loss: 0.8843 - accuracy: 0.6837
Epoch 223/250
784/784 [==============================] - 0s 28us/step - loss: 0.8789 - accuracy: 0.6760
Epoch 224/250
784/784 [==============================] - 0s 33us/step - loss: 0.8718 - accuracy: 0.6977
Epoch 225/250
784/784 [==============================] - 0s 37us/step - loss: 0.8769 - accuracy: 0.6862
Epoch 226/250
784/784 [==============================] - ETA: 0s - loss: 0.8089 - accuracy: 0.74 - 0s 39us/step - loss: 0.8752 - accuracy: 0.6901
Epoch 227/250
784/784 [==============================] - 0s 39us/step - loss: 0.8850 - accuracy: 0.6875
Epoch 228/250
784/784 [==============================] - 0s 36us/step - loss: 0.8607 - accuracy: 0.6939
Epoch 229/250
784/784 [=============

784/784 [==============================] - 0s 29us/step - loss: 1.0636 - accuracy: 0.6378
Epoch 127/250
784/784 [==============================] - 0s 29us/step - loss: 1.0773 - accuracy: 0.6097
Epoch 128/250
784/784 [==============================] - 0s 27us/step - loss: 1.0499 - accuracy: 0.6441
Epoch 129/250
784/784 [==============================] - 0s 27us/step - loss: 1.0360 - accuracy: 0.6505
Epoch 130/250
784/784 [==============================] - 0s 28us/step - loss: 1.0526 - accuracy: 0.6620
Epoch 131/250
784/784 [==============================] - 0s 28us/step - loss: 1.0349 - accuracy: 0.6505
Epoch 132/250
784/784 [==============================] - 0s 28us/step - loss: 1.0580 - accuracy: 0.6467
Epoch 133/250
784/784 [==============================] - 0s 28us/step - loss: 1.0188 - accuracy: 0.6505
Epoch 134/250
784/784 [==============================] - 0s 29us/step - loss: 1.0246 - accuracy: 0.6709
Epoch 135/250
784/784 [==============================] - 0s 29us/step - loss: 

...

784/784 [==============================] - 0s 42us/step - loss: 1.1889 - accuracy: 0.5612
Epoch 107/250
784/784 [==============================] - 0s 42us/step - loss: 1.1529 - accuracy: 0.5625
Epoch 108/250
784/784 [==============================] - 0s 45us/step - loss: 1.1390 - accuracy: 0.5957
Epoch 109/250
784/784 [==============================] - 0s 38us/step - loss: 1.1660 - accuracy: 0.5804
Epoch 110/250
784/784 [==============================] - 0s 37us/step - loss: 1.1339 - accuracy: 0.5906
Epoch 111/250
784/784 [==============================] - 0s 34us/step - loss: 1.1595 - accuracy: 0.5727
Epoch 112/250
784/784 [==============================] - 0s 37us/step - loss: 1.1520 - accuracy: 0.5957
Epoch 113/250
784/784 [==============================] - 0s 38us/step - loss: 1.1342 - accuracy: 0.5931
Epoch 114/250
784/784 [==============================] - 0s 32us/step - loss: 1.1406 - accuracy: 0.5689
Epoch 115/250
784/784 [==============================] - 0s 32us/step - loss: 

784/784 [==============================] - 0s 37us/step - loss: 2.2797 - accuracy: 0.2015
Epoch 10/250
784/784 [==============================] - 0s 39us/step - loss: 2.2717 - accuracy: 0.1990
Epoch 11/250
784/784 [==============================] - 0s 38us/step - loss: 2.2637 - accuracy: 0.2054
Epoch 12/250
784/784 [==============================] - 0s 39us/step - loss: 2.2528 - accuracy: 0.2054
Epoch 13/250
784/784 [==============================] - 0s 38us/step - loss: 2.2367 - accuracy: 0.2130
Epoch 14/250
784/784 [==============================] - 0s 46us/step - loss: 2.2252 - accuracy: 0.2041
Epoch 15/250
784/784 [==============================] - 0s 41us/step - loss: 2.2079 - accuracy: 0.2105
Epoch 16/250
784/784 [==============================] - 0s 45us/step - loss: 2.1851 - accuracy: 0.2130
Epoch 17/250
784/784 [==============================] - 0s 41us/step - loss: 2.1669 - accuracy: 0.2347
Epoch 18/250
784/784 [==============================] - 0s 38us/step - loss: 2.1458 - 

784/784 [==============================] - 0s 36us/step - loss: 0.8793 - accuracy: 0.7117
Epoch 165/250
784/784 [==============================] - 0s 36us/step - loss: 0.8871 - accuracy: 0.7168
Epoch 166/250
784/784 [==============================] - 0s 34us/step - loss: 0.9062 - accuracy: 0.7054
Epoch 167/250
784/784 [==============================] - 0s 34us/step - loss: 0.8794 - accuracy: 0.7066
Epoch 168/250
784/784 [==============================] - 0s 32us/step - loss: 0.8844 - accuracy: 0.7092
Epoch 169/250
784/784 [==============================] - 0s 34us/step - loss: 0.8786 - accuracy: 0.7079
Epoch 170/250
784/784 [==============================] - 0s 33us/step - loss: 0.8708 - accuracy: 0.7245
Epoch 171/250
784/784 [==============================] - 0s 33us/step - loss: 0.8746 - accuracy: 0.7117
Epoch 172/250
784/784 [==============================] - 0s 33us/step - loss: 0.8873 - accuracy: 0.7181
Epoch 173/250
784/784 [==============================] - 0s 34us/step - loss: 

784/784 [==============================] - 0s 32us/step - loss: 0.6033 - accuracy: 0.8023
Epoch 243/250
784/784 [==============================] - 0s 34us/step - loss: 0.6090 - accuracy: 0.7985
Epoch 244/250
784/784 [==============================] - 0s 31us/step - loss: 0.6055 - accuracy: 0.8087
Epoch 245/250
784/784 [==============================] - 0s 33us/step - loss: 0.6059 - accuracy: 0.8087
Epoch 246/250
784/784 [==============================] - 0s 33us/step - loss: 0.6147 - accuracy: 0.8010
Epoch 247/250
784/784 [==============================] - 0s 34us/step - loss: 0.5828 - accuracy: 0.8099
Epoch 248/250
784/784 [==============================] - 0s 32us/step - loss: 0.6062 - accuracy: 0.7921
Epoch 249/250
784/784 [==============================] - 0s 34us/step - loss: 0.5791 - accuracy: 0.8074
Epoch 250/250
196/196 [==============================] - 0s 1ms/step


0.7285714268684387

Model 3

In [139]:
# for deskewed
evaluate_classifier(build_model_3, nn_train_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 406us/step - loss: 2.3026 - accuracy: 0.0995
Epoch 2/250
784/784 [==============================] - 0s 38us/step - loss: 2.2995 - accuracy: 0.1199
Epoch 3/250
784/784 [==============================] - 0s 41us/step - loss: 2.2945 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 39us/step - loss: 2.2873 - accuracy: 0.1365
Epoch 5/250
784/784 [==============================] - 0s 39us/step - loss: 2.2701 - accuracy: 0.1543
Epoch 6/250
784/784 [==============================] - 0s 37us/step - loss: 2.2455 - accuracy: 0.1684
Epoch 7/250
784/784 [==============================] - ETA: 0s - loss: 2.2322 - accuracy: 0.20 - 0s 41us/step - loss: 2.2162 - accuracy: 0.1722
Epoch 8/250
784/784 [==============================] - 0s 41us/step - loss: 2.1551 - accuracy: 0.1888
Epoch 9/250
784/784 [==============================] - 0s 41us/step - loss: 2.1030 - accuracy: 0.1875
Epoch 10/250
784/78

784/784 [==============================] - 0s 39us/step - loss: 1.0950 - accuracy: 0.5638
Epoch 156/250
784/784 [==============================] - 0s 42us/step - loss: 1.0348 - accuracy: 0.5893
Epoch 157/250
784/784 [==============================] - 0s 42us/step - loss: 1.0249 - accuracy: 0.5867
Epoch 158/250
784/784 [==============================] - 0s 43us/step - loss: 0.9656 - accuracy: 0.6288
Epoch 159/250
784/784 [==============================] - 0s 42us/step - loss: 0.9983 - accuracy: 0.5829
Epoch 160/250
784/784 [==============================] - 0s 43us/step - loss: 1.1365 - accuracy: 0.5536
Epoch 161/250
784/784 [==============================] - 0s 45us/step - loss: 0.9782 - accuracy: 0.6492
Epoch 162/250
784/784 [==============================] - 0s 45us/step - loss: 0.9455 - accuracy: 0.6492
Epoch 163/250
784/784 [==============================] - 0s 41us/step - loss: 1.0322 - accuracy: 0.5867
Epoch 164/250
784/784 [==============================] - 0s 37us/step - loss: 

784/784 [==============================] - 0s 41us/step - loss: 1.1477 - accuracy: 0.5421
Epoch 61/250
784/784 [==============================] - ETA: 0s - loss: 1.1572 - accuracy: 0.52 - 0s 39us/step - loss: 1.1363 - accuracy: 0.5344
Epoch 62/250
784/784 [==============================] - 0s 38us/step - loss: 1.1512 - accuracy: 0.5383
Epoch 63/250
784/784 [==============================] - 0s 37us/step - loss: 1.0646 - accuracy: 0.5791
Epoch 64/250
784/784 [==============================] - 0s 36us/step - loss: 1.0578 - accuracy: 0.5663
Epoch 65/250
784/784 [==============================] - 0s 37us/step - loss: 1.0785 - accuracy: 0.5587
Epoch 66/250
784/784 [==============================] - 0s 37us/step - loss: 1.1119 - accuracy: 0.5485
Epoch 67/250
784/784 [==============================] - 0s 37us/step - loss: 1.0749 - accuracy: 0.5842
Epoch 68/250
784/784 [==============================] - 0s 36us/step - loss: 1.1216 - accuracy: 0.5166
Epoch 69/250
784/784 [======================

784/784 [==============================] - 0s 37us/step - loss: 0.9112 - accuracy: 0.6276
Epoch 139/250
784/784 [==============================] - 0s 34us/step - loss: 0.7980 - accuracy: 0.7092
Epoch 140/250
784/784 [==============================] - 0s 36us/step - loss: 0.9011 - accuracy: 0.6276
Epoch 141/250
784/784 [==============================] - 0s 34us/step - loss: 0.8363 - accuracy: 0.6658
Epoch 142/250
784/784 [==============================] - 0s 36us/step - loss: 0.8289 - accuracy: 0.6901
Epoch 143/250
784/784 [==============================] - 0s 36us/step - loss: 0.9445 - accuracy: 0.5995
Epoch 144/250
784/784 [==============================] - 0s 34us/step - loss: 0.7851 - accuracy: 0.6952
Epoch 145/250
784/784 [==============================] - 0s 36us/step - loss: 0.7703 - accuracy: 0.7054
Epoch 146/250
784/784 [==============================] - 0s 36us/step - loss: 0.9463 - accuracy: 0.5931
Epoch 147/250
784/784 [==============================] - 0s 36us/step - loss: 

784/784 [==============================] - 0s 39us/step - loss: 1.5598 - accuracy: 0.3903
Epoch 42/250
784/784 [==============================] - 0s 38us/step - loss: 1.6486 - accuracy: 0.3291
Epoch 43/250
784/784 [==============================] - 0s 38us/step - loss: 1.5507 - accuracy: 0.3393
Epoch 44/250
784/784 [==============================] - 0s 34us/step - loss: 1.6302 - accuracy: 0.3163
Epoch 45/250
784/784 [==============================] - 0s 34us/step - loss: 1.5624 - accuracy: 0.3699
Epoch 46/250
784/784 [==============================] - 0s 36us/step - loss: 1.5644 - accuracy: 0.3699
Epoch 47/250
784/784 [==============================] - 0s 38us/step - loss: 1.6413 - accuracy: 0.3125
Epoch 48/250
784/784 [==============================] - 0s 38us/step - loss: 1.5705 - accuracy: 0.3546
Epoch 49/250
784/784 [==============================] - 0s 37us/step - loss: 1.5270 - accuracy: 0.3865
Epoch 50/250
784/784 [==============================] - 0s 36us/step - loss: 1.6426 - 

Epoch 197/250
784/784 [==============================] - 0s 39us/step - loss: 1.0005 - accuracy: 0.6582
Epoch 198/250
784/784 [==============================] - 0s 47us/step - loss: 1.0896 - accuracy: 0.5982
Epoch 199/250
784/784 [==============================] - 0s 43us/step - loss: 1.0319 - accuracy: 0.6633
Epoch 200/250
784/784 [==============================] - 0s 46us/step - loss: 0.9594 - accuracy: 0.6684
Epoch 201/250
784/784 [==============================] - 0s 45us/step - loss: 1.0265 - accuracy: 0.6378
Epoch 202/250
784/784 [==============================] - 0s 46us/step - loss: 0.9446 - accuracy: 0.6620
Epoch 203/250
784/784 [==============================] - 0s 45us/step - loss: 1.0307 - accuracy: 0.6454
Epoch 204/250
784/784 [==============================] - 0s 39us/step - loss: 0.9575 - accuracy: 0.6824
Epoch 205/250
784/784 [==============================] - 0s 38us/step - loss: 0.9619 - accuracy: 0.6939
Epoch 206/250
784/784 [==============================] - 0s 37us

784/784 [==============================] - 0s 33us/step - loss: 1.2956 - accuracy: 0.4809
Epoch 100/250
784/784 [==============================] - 0s 41us/step - loss: 1.3818 - accuracy: 0.4120
Epoch 101/250
784/784 [==============================] - 0s 42us/step - loss: 1.2901 - accuracy: 0.4872
Epoch 102/250
784/784 [==============================] - 0s 39us/step - loss: 1.4332 - accuracy: 0.4056
Epoch 103/250
784/784 [==============================] - 0s 41us/step - loss: 1.3959 - accuracy: 0.4145
Epoch 104/250
784/784 [==============================] - 0s 38us/step - loss: 1.2925 - accuracy: 0.4911
Epoch 105/250
784/784 [==============================] - 0s 39us/step - loss: 1.3712 - accuracy: 0.4209
Epoch 106/250
784/784 [==============================] - 0s 39us/step - loss: 1.3031 - accuracy: 0.4643
Epoch 107/250
784/784 [==============================] - 0s 39us/step - loss: 1.3612 - accuracy: 0.4464
Epoch 108/250
784/784 [==============================] - ETA: 0s - loss: 1.361

784/784 [==============================] - 0s 36us/step - loss: 2.3017 - accuracy: 0.1110
Epoch 3/250
784/784 [==============================] - 0s 41us/step - loss: 2.2990 - accuracy: 0.1378
Epoch 4/250
784/784 [==============================] - 0s 34us/step - loss: 2.2918 - accuracy: 0.1543
Epoch 5/250
784/784 [==============================] - 0s 33us/step - loss: 2.2827 - accuracy: 0.1518
Epoch 6/250
784/784 [==============================] - 0s 34us/step - loss: 2.2610 - accuracy: 0.1722
Epoch 7/250
784/784 [==============================] - 0s 34us/step - loss: 2.2200 - accuracy: 0.1888
Epoch 8/250
784/784 [==============================] - 0s 38us/step - loss: 2.1850 - accuracy: 0.1901
Epoch 9/250
784/784 [==============================] - 0s 33us/step - loss: 2.1235 - accuracy: 0.2105
Epoch 10/250
784/784 [==============================] - 0s 32us/step - loss: 2.0823 - accuracy: 0.2194
Epoch 11/250
784/784 [==============================] - 0s 34us/step - loss: 2.0546 - accurac

784/784 [==============================] - 0s 38us/step - loss: 1.1867 - accuracy: 0.4962
Epoch 159/250
784/784 [==============================] - 0s 39us/step - loss: 1.2063 - accuracy: 0.5268
Epoch 160/250
784/784 [==============================] - 0s 38us/step - loss: 1.2687 - accuracy: 0.4643
Epoch 161/250
784/784 [==============================] - 0s 38us/step - loss: 1.1379 - accuracy: 0.5383
Epoch 162/250
784/784 [==============================] - 0s 37us/step - loss: 1.2608 - accuracy: 0.4605
Epoch 163/250
784/784 [==============================] - 0s 41us/step - loss: 1.1888 - accuracy: 0.5179
Epoch 164/250
784/784 [==============================] - 0s 39us/step - loss: 1.2169 - accuracy: 0.4898
Epoch 165/250
784/784 [==============================] - 0s 41us/step - loss: 1.2446 - accuracy: 0.4834
Epoch 166/250
784/784 [==============================] - 0s 36us/step - loss: 1.1824 - accuracy: 0.4860
Epoch 167/250
784/784 [==============================] - 0s 37us/step - loss: 

0.7071428537368775

In [140]:
# for non deskewed
evaluate_classifier(build_model_3, nn_train_non_deskewed, encoded_train_labels, 250, scoring='accuracy')

processing fold # 0
Epoch 1/250
784/784 [==============================] - 0s 463us/step - loss: 2.3035 - accuracy: 0.1071
Epoch 2/250
784/784 [==============================] - 0s 39us/step - loss: 2.3006 - accuracy: 0.1199
Epoch 3/250
784/784 [==============================] - 0s 46us/step - loss: 2.2978 - accuracy: 0.1199
Epoch 4/250
784/784 [==============================] - 0s 42us/step - loss: 2.2928 - accuracy: 0.1339
Epoch 5/250
784/784 [==============================] - 0s 39us/step - loss: 2.2812 - accuracy: 0.1378
Epoch 6/250
784/784 [==============================] - 0s 39us/step - loss: 2.2597 - accuracy: 0.1607
Epoch 7/250
784/784 [==============================] - 0s 38us/step - loss: 2.2378 - accuracy: 0.1849
Epoch 8/250
784/784 [==============================] - 0s 38us/step - loss: 2.1842 - accuracy: 0.1977
Epoch 9/250
784/784 [==============================] - 0s 39us/step - loss: 2.1571 - accuracy: 0.2003
Epoch 10/250
784/784 [==============================] - 0s 39

784/784 [==============================] - 0s 39us/step - loss: 1.1631 - accuracy: 0.5383
Epoch 155/250
784/784 [==============================] - 0s 39us/step - loss: 1.0440 - accuracy: 0.6020
Epoch 156/250
784/784 [==============================] - 0s 37us/step - loss: 1.1206 - accuracy: 0.5421
Epoch 157/250
784/784 [==============================] - 0s 38us/step - loss: 1.1273 - accuracy: 0.5804
Epoch 158/250
784/784 [==============================] - 0s 38us/step - loss: 1.0955 - accuracy: 0.5753
Epoch 159/250
784/784 [==============================] - 0s 39us/step - loss: 1.0789 - accuracy: 0.5829
Epoch 160/250
784/784 [==============================] - 0s 37us/step - loss: 1.0744 - accuracy: 0.5829
Epoch 161/250
784/784 [==============================] - 0s 37us/step - loss: 1.1546 - accuracy: 0.5587
Epoch 162/250
784/784 [==============================] - 0s 37us/step - loss: 1.0326 - accuracy: 0.6135
Epoch 163/250
784/784 [==============================] - 0s 36us/step - loss: 

784/784 [==============================] - 0s 37us/step - loss: 1.2735 - accuracy: 0.4707
Epoch 58/250
784/784 [==============================] - 0s 36us/step - loss: 1.1641 - accuracy: 0.5128
Epoch 59/250
784/784 [==============================] - 0s 39us/step - loss: 1.2712 - accuracy: 0.4923
Epoch 60/250
784/784 [==============================] - 0s 45us/step - loss: 1.1864 - accuracy: 0.5038
Epoch 61/250
784/784 [==============================] - 0s 43us/step - loss: 1.1556 - accuracy: 0.5179
Epoch 62/250
784/784 [==============================] - 0s 45us/step - loss: 1.2413 - accuracy: 0.4528
Epoch 63/250
784/784 [==============================] - 0s 45us/step - loss: 1.1550 - accuracy: 0.5408
Epoch 64/250
784/784 [==============================] - ETA: 0s - loss: 1.1213 - accuracy: 0.52 - 0s 38us/step - loss: 1.1977 - accuracy: 0.5038
Epoch 65/250
784/784 [==============================] - 0s 37us/step - loss: 1.1600 - accuracy: 0.5344
Epoch 66/250
784/784 [======================

784/784 [==============================] - 0s 37us/step - loss: 0.8016 - accuracy: 0.6901
Epoch 212/250
784/784 [==============================] - 0s 37us/step - loss: 0.9698 - accuracy: 0.6288
Epoch 213/250
784/784 [==============================] - 0s 36us/step - loss: 0.8131 - accuracy: 0.6735
Epoch 214/250
784/784 [==============================] - 0s 36us/step - loss: 0.9980 - accuracy: 0.5855
Epoch 215/250
784/784 [==============================] - 0s 37us/step - loss: 0.7745 - accuracy: 0.7232
Epoch 216/250
784/784 [==============================] - 0s 38us/step - loss: 0.8765 - accuracy: 0.6620
Epoch 217/250
784/784 [==============================] - 0s 37us/step - loss: 0.8906 - accuracy: 0.6658
Epoch 218/250
784/784 [==============================] - 0s 37us/step - loss: 0.8187 - accuracy: 0.7003
Epoch 219/250
784/784 [==============================] - 0s 37us/step - loss: 0.9089 - accuracy: 0.6569
Epoch 220/250
784/784 [==============================] - ETA: 0s - loss: 0.729

784/784 [==============================] - 0s 41us/step - loss: 1.1247 - accuracy: 0.5574
Epoch 114/250
784/784 [==============================] - 0s 45us/step - loss: 1.0538 - accuracy: 0.6008
Epoch 115/250
784/784 [==============================] - 0s 46us/step - loss: 1.1088 - accuracy: 0.5791
Epoch 116/250
784/784 [==============================] - 0s 45us/step - loss: 1.0513 - accuracy: 0.5995
Epoch 117/250
784/784 [==============================] - 0s 42us/step - loss: 1.0460 - accuracy: 0.5918
Epoch 118/250
784/784 [==============================] - ETA: 0s - loss: 1.0809 - accuracy: 0.57 - 0s 41us/step - loss: 1.0133 - accuracy: 0.6084
Epoch 119/250
784/784 [==============================] - 0s 38us/step - loss: 1.0236 - accuracy: 0.5982
Epoch 120/250
784/784 [==============================] - 0s 37us/step - loss: 1.0729 - accuracy: 0.6046
Epoch 121/250
784/784 [==============================] - 0s 43us/step - loss: 0.9719 - accuracy: 0.6505
Epoch 122/250
784/784 [=============

Epoch 15/250
784/784 [==============================] - 0s 39us/step - loss: 1.9716 - accuracy: 0.2679
Epoch 16/250
784/784 [==============================] - 0s 39us/step - loss: 1.9592 - accuracy: 0.2628
Epoch 17/250
784/784 [==============================] - 0s 37us/step - loss: 1.9372 - accuracy: 0.2538
Epoch 18/250
784/784 [==============================] - 0s 37us/step - loss: 1.8855 - accuracy: 0.2934
Epoch 19/250
784/784 [==============================] - 0s 38us/step - loss: 1.8962 - accuracy: 0.2487
Epoch 20/250
784/784 [==============================] - 0s 38us/step - loss: 1.8500 - accuracy: 0.3099
Epoch 21/250
784/784 [==============================] - 0s 39us/step - loss: 1.8380 - accuracy: 0.3074
Epoch 22/250
784/784 [==============================] - 0s 37us/step - loss: 1.8538 - accuracy: 0.2870
Epoch 23/250
784/784 [==============================] - 0s 36us/step - loss: 1.8114 - accuracy: 0.3048
Epoch 24/250
784/784 [==============================] - 0s 36us/step - lo

784/784 [==============================] - 0s 41us/step - loss: 0.8935 - accuracy: 0.6276
Epoch 170/250
784/784 [==============================] - 0s 41us/step - loss: 1.0168 - accuracy: 0.5995
Epoch 171/250
784/784 [==============================] - 0s 39us/step - loss: 1.0026 - accuracy: 0.5906
Epoch 172/250
784/784 [==============================] - ETA: 0s - loss: 1.0914 - accuracy: 0.56 - 0s 39us/step - loss: 0.9059 - accuracy: 0.6505
Epoch 173/250
784/784 [==============================] - 0s 42us/step - loss: 0.9164 - accuracy: 0.6110
Epoch 174/250
784/784 [==============================] - 0s 41us/step - loss: 0.9055 - accuracy: 0.6250
Epoch 175/250
784/784 [==============================] - 0s 42us/step - loss: 1.0331 - accuracy: 0.6084
Epoch 176/250
784/784 [==============================] - 0s 41us/step - loss: 0.9144 - accuracy: 0.6186
Epoch 177/250
784/784 [==============================] - 0s 42us/step - loss: 0.8916 - accuracy: 0.6594
Epoch 178/250
784/784 [=============

784/784 [==============================] - 0s 45us/step - loss: 1.5751 - accuracy: 0.3533
Epoch 72/250
784/784 [==============================] - 0s 43us/step - loss: 1.5180 - accuracy: 0.3967
Epoch 73/250
784/784 [==============================] - 0s 41us/step - loss: 1.5829 - accuracy: 0.3635
Epoch 74/250
784/784 [==============================] - 0s 43us/step - loss: 1.4996 - accuracy: 0.4145
Epoch 75/250
784/784 [==============================] - 0s 43us/step - loss: 1.6351 - accuracy: 0.3367
Epoch 76/250
784/784 [==============================] - 0s 41us/step - loss: 1.4546 - accuracy: 0.4158
Epoch 77/250
784/784 [==============================] - 0s 42us/step - loss: 1.5739 - accuracy: 0.3584
Epoch 78/250
784/784 [==============================] - 0s 39us/step - loss: 1.5188 - accuracy: 0.4043
Epoch 79/250
784/784 [==============================] - 0s 41us/step - loss: 1.4877 - accuracy: 0.3801
Epoch 80/250
784/784 [==============================] - 0s 42us/step - loss: 1.6012 - 

784/784 [==============================] - 0s 42us/step - loss: 0.8790 - accuracy: 0.6582
Epoch 224/250
784/784 [==============================] - ETA: 0s - loss: 0.9358 - accuracy: 0.62 - 0s 46us/step - loss: 0.8347 - accuracy: 0.6709
Epoch 225/250
784/784 [==============================] - 0s 45us/step - loss: 0.8040 - accuracy: 0.6862
Epoch 226/250
784/784 [==============================] - 0s 46us/step - loss: 0.8833 - accuracy: 0.6735
Epoch 227/250
784/784 [==============================] - 0s 46us/step - loss: 0.8259 - accuracy: 0.6798
Epoch 228/250
784/784 [==============================] - 0s 41us/step - loss: 0.7746 - accuracy: 0.7066
Epoch 229/250
784/784 [==============================] - 0s 42us/step - loss: 0.9141 - accuracy: 0.6620
Epoch 230/250
784/784 [==============================] - 0s 41us/step - loss: 0.7524 - accuracy: 0.7245
Epoch 231/250
784/784 [==============================] - 0s 43us/step - loss: 0.8616 - accuracy: 0.7003
Epoch 232/250
784/784 [=============

0.6826530694961548

# Training models

Dla obu danych najlepszy okazał się model 2gi

In [141]:
network_deskewed = build_model_2()
network_non_deskewed = build_model_2()

In [142]:
network_deskewed.fit(nn_train_deskewed, encoded_train_labels, epochs=300, batch_size=128)

Epoch 1/300
980/980 [==============================] - 0s 401us/step - loss: 2.3027 - accuracy: 0.0888
Epoch 2/300
980/980 [==============================] - 0s 34us/step - loss: 2.3009 - accuracy: 0.1122
Epoch 3/300
980/980 [==============================] - 0s 37us/step - loss: 2.2983 - accuracy: 0.1194
Epoch 4/300
980/980 [==============================] - 0s 35us/step - loss: 2.2958 - accuracy: 0.1194
Epoch 5/300
980/980 [==============================] - 0s 35us/step - loss: 2.2905 - accuracy: 0.1520
Epoch 6/300
980/980 [==============================] - 0s 37us/step - loss: 2.2833 - accuracy: 0.1429
Epoch 7/300
980/980 [==============================] - 0s 35us/step - loss: 2.2743 - accuracy: 0.1827
Epoch 8/300
980/980 [==============================] - 0s 32us/step - loss: 2.2630 - accuracy: 0.1663
Epoch 9/300
980/980 [==============================] - 0s 38us/step - loss: 2.2471 - accuracy: 0.1929
Epoch 10/300
980/980 [==============================] - 0s 39us/step - loss: 2.22

980/980 [==============================] - 0s 36us/step - loss: 0.7953 - accuracy: 0.7398
Epoch 156/300
980/980 [==============================] - 0s 35us/step - loss: 0.7937 - accuracy: 0.7306
Epoch 157/300
980/980 [==============================] - 0s 37us/step - loss: 0.7926 - accuracy: 0.7347
Epoch 158/300
980/980 [==============================] - 0s 37us/step - loss: 0.7652 - accuracy: 0.7551
Epoch 159/300
980/980 [==============================] - 0s 36us/step - loss: 0.7720 - accuracy: 0.7367
Epoch 160/300
980/980 [==============================] - 0s 38us/step - loss: 0.7963 - accuracy: 0.7296
Epoch 161/300
980/980 [==============================] - 0s 37us/step - loss: 0.7722 - accuracy: 0.7439
Epoch 162/300
980/980 [==============================] - 0s 36us/step - loss: 0.7727 - accuracy: 0.7439
Epoch 163/300
980/980 [==============================] - 0s 36us/step - loss: 0.7759 - accuracy: 0.7439
Epoch 164/300
980/980 [==============================] - 0s 35us/step - loss: 

In [143]:
network_non_deskewed.fit(nn_train_non_deskewed, encoded_train_labels, epochs=300, batch_size=128)

Epoch 1/300
980/980 [==============================] - 0s 389us/step - loss: 2.3032 - accuracy: 0.1194
Epoch 2/300
980/980 [==============================] - 0s 33us/step - loss: 2.3006 - accuracy: 0.1316
Epoch 3/300
980/980 [==============================] - 0s 33us/step - loss: 2.2984 - accuracy: 0.1388
Epoch 4/300
980/980 [==============================] - 0s 33us/step - loss: 2.2950 - accuracy: 0.1622
Epoch 5/300
980/980 [==============================] - 0s 33us/step - loss: 2.2905 - accuracy: 0.1990
Epoch 6/300
980/980 [==============================] - 0s 36us/step - loss: 2.2831 - accuracy: 0.1735
Epoch 7/300
980/980 [==============================] - 0s 41us/step - loss: 2.2749 - accuracy: 0.1959
Epoch 8/300
980/980 [==============================] - 0s 36us/step - loss: 2.2632 - accuracy: 0.1949
Epoch 9/300
980/980 [==============================] - 0s 36us/step - loss: 2.2491 - accuracy: 0.2010
Epoch 10/300
980/980 [==============================] - 0s 37us/step - loss: 2.23

980/980 [==============================] - 0s 34us/step - loss: 0.8030 - accuracy: 0.7439
Epoch 157/300
980/980 [==============================] - 0s 30us/step - loss: 0.8184 - accuracy: 0.7500
Epoch 158/300
980/980 [==============================] - 0s 30us/step - loss: 0.7899 - accuracy: 0.7551
Epoch 159/300
980/980 [==============================] - 0s 31us/step - loss: 0.8152 - accuracy: 0.7327
Epoch 160/300
980/980 [==============================] - 0s 32us/step - loss: 0.8068 - accuracy: 0.7367
Epoch 161/300
980/980 [==============================] - 0s 31us/step - loss: 0.7790 - accuracy: 0.7439
Epoch 162/300
980/980 [==============================] - 0s 32us/step - loss: 0.7863 - accuracy: 0.7490
Epoch 163/300
980/980 [==============================] - 0s 33us/step - loss: 0.7740 - accuracy: 0.7480
Epoch 164/300
980/980 [==============================] - 0s 32us/step - loss: 0.7824 - accuracy: 0.7378
Epoch 165/300
980/980 [==============================] - 0s 31us/step - loss: 

In [144]:
pred_probabilities_deskewed = network_deskewed.predict(nn_test_deskewed)
pred_probabilities_non_deskewed = network_non_deskewed.predict(nn_test_non_deskewed)

pred_labels_deskewed = np.argmax(pred_probabilities_deskewed,-1)
pred_labels_non_deskewed = np.argmax(pred_probabilities_non_deskewed,-1)

In [145]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy score for deskewed data: {}\nAccuracy score for non deskewed data: {}".\
     format(accuracy_score(test_labels, pred_labels_deskewed), accuracy_score(test_labels, pred_labels_non_deskewed)))

Accuracy score for deskewed data: 0.8502752825268038
Accuracy score for non deskewed data: 0.8488264271225732


In [146]:
cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)
print("Confusion matrix for deskewed data: \n{}\n\nConfusion matrix for non deskewed data: \n{}".\
      format(cm_deskewed, cm_non_deskewed))

Confusion matrix for deskewed data: 
[[5693   19   90    1  198   19  185   38   75  488]
 [ 152 7322  139    0   73    0    2   50   27    2]
 [  35   84 5591  348   20    1    0  359  443   11]
 [   4    1  254 6118    0   86    0  113  449   16]
 [ 136   31    5    0 5879    6  444    5  117  105]
 [   1    0    3   70   20 5615   62    2  321  131]
 [  36   12    3    0  183   47 6466    0   31    2]
 [ 133    3  371  244   10    0    0 6084   54  292]
 [  47   15  113  219  140  580   36   25 5075  479]
 [ 593    2    9   42   63  422   13  169  705 4843]]

Confusion matrix for non deskewed data: 
[[5919   14  101    2   32   22  175   65  156  320]
 [  30 7316  134    4   23    0    5   92  158    5]
 [  13   96 5324  570    3    0    1  610  269    6]
 [   2    0  303 6229    2   81    0  201  200   23]
 [ 126   16    4    2 5877   20  183   12  244  244]
 [  15    5   16  209   57 5085   28    6  494  310]
 [ 273   70    2    2  162   80 6102    0   83    6]
 [  38    0  333  2

In [147]:
cr_deskewed = classification_report(test_labels, pred_labels_deskewed)
cr_non_deskewed = classification_report(test_labels, pred_labels_non_deskewed)

print("Clasification report for deskewed data: \n{}\n\nClassification report for non deskewed data: \n{}".\
     format(cr_deskewed, cr_non_deskewed))


Clasification report for deskewed data: 
             precision    recall  f1-score   support

          0       0.83      0.84      0.83      6806
          1       0.98      0.94      0.96      7767
          2       0.85      0.81      0.83      6892
          3       0.87      0.87      0.87      7041
          4       0.89      0.87      0.88      6728
          5       0.83      0.90      0.86      6225
          6       0.90      0.95      0.92      6780
          7       0.89      0.85      0.87      7191
          8       0.70      0.75      0.72      6729
          9       0.76      0.71      0.73      6861

avg / total       0.85      0.85      0.85     69020


Classification report for non deskewed data: 
             precision    recall  f1-score   support

          0       0.87      0.87      0.87      6806
          1       0.96      0.94      0.95      7767
          2       0.83      0.77      0.80      6892
          3       0.80      0.88      0.84      7041
       

# Conclusion
Deskewing images before applying HOG descriptor increases performance for every of tested models.